<a href="https://colab.research.google.com/github/amimulhasan/Deep-Learning/blob/main/vit_gru.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [9]:
!kaggle datasets download mohammadhossein77/brain-tumors-dataset

Dataset URL: https://www.kaggle.com/datasets/mohammadhossein77/brain-tumors-dataset
License(s): CC0-1.0
brain-tumors-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [10]:
import zipfile

zip_path = '/content/brain-tumors-dataset.zip'
extract_to = 'brain_tumor_data'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("Unzipping completed!")

Unzipping completed!


In [11]:
import os, random
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import timm
from sklearn.metrics import accuracy_score, classification_report


In [12]:
class FlatTumorDataset(Dataset):
    def __init__(self, root_dir, transform, sequence_len=5):
        self.transform = transform
        self.sequence_len = sequence_len
        self.samples = []
        self.label_map = {}
        for idx, label in enumerate(sorted(os.listdir(root_dir))):
            label_path = os.path.join(root_dir, label)
            if not os.path.isdir(label_path): continue
            self.label_map[label] = idx
            files = sorted([os.path.join(label_path, f) for f in os.listdir(label_path) if f.endswith(('.jpg', '.png'))])
            for i in range(0, len(files) - sequence_len + 1, sequence_len):
                self.samples.append((files[i:i+sequence_len], idx))

    def __len__(self): return len(self.samples)

    def __getitem__(self, idx):
        paths, label = self.samples[idx]
        images = [self.transform(Image.open(p).convert('RGB')) for p in paths]
        return torch.stack(images), label


In [13]:
class ViT_GRU_Model(nn.Module):
    def __init__(self, vit_model='vit_base_patch16_224', hidden_size=256, num_classes=3):
        super().__init__()
        self.vit = timm.create_model(vit_model, pretrained=True)
        self.vit.head = nn.Identity()
        self.embed_dim = self.vit.embed_dim

        self.gru = nn.GRU(self.embed_dim, hidden_size, batch_first=True, bidirectional=True)
        self.fc = nn.Sequential(
            nn.Linear(hidden_size*2, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        B, T, C, H, W = x.shape
        x = x.view(B*T, C, H, W)
        feat = self.vit(x)
        feat = feat.view(B, T, -1)
        gru_out, _ = self.gru(feat)
        return self.fc(gru_out[:, -1, :])


In [14]:
def train_epoch(model, loader, optimizer, loss_fn, device):
    model.train()
    total_loss = 0
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        out = model(x)
        loss = loss_fn(out, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

def evaluate(model, loader, device, label_map):
    model.eval()
    preds, targets = [], []
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            out = model(x)
            pred = out.argmax(dim=1).cpu().tolist()
            preds.extend(pred)
            targets.extend(y.tolist())
    acc = accuracy_score(targets, preds)
    report = classification_report(targets, preds, target_names=list(label_map.keys()))
    return acc, report


In [ ]:
# Configuration
root = "/content/brain_tumor_data/Data"
transform = transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor()])
sequence_len = 5
batch_size = 4
epochs = 5
lr = 1e-4

# Dataset and loaders
dataset = FlatTumorDataset(root, transform, sequence_len)
random.shuffle(dataset.samples)
split = int(0.8 * len(dataset))
train_set = torch.utils.data.Subset(dataset, range(split))
test_set = torch.utils.data.Subset(dataset, range(split, len(dataset)))
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

# Model setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ViT_GRU_Model(num_classes=len(dataset.label_map)).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_fn = nn.CrossEntropyLoss()

# Training loop
for epoch in range(1, epochs+1):
    train_loss = train_epoch(model, train_loader, optimizer, loss_fn, device)
    acc, report = evaluate(model, test_loader, device, dataset.label_map)
    print(f"Epoch {epoch}/{epochs} | Loss: {train_loss:.4f} | Accuracy: {acc:.4f}")

# Final evaluation report
print("\n📊 Classification Report:\n", report)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]